In [17]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import tabulate as tb

In [ ]:
TICKER = "GOOGL"
PATH = f'./../data/{TICKER}_1h.csv'
START_DATE = "2023-01-01"
END_DATE = None
TIMEFRAME = "1h"    # 1d, 1h, 15m, 5m

In [19]:
df = yf.download(
    TICKER, 
    start=START_DATE,
    end=END_DATE,
    interval=TIMEFRAME,
    progress=False
)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

df.dropna(inplace=True)
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

/tmp/ipykernel_76024/3251543518.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(

1 Failed download:
['GOOGL']: YFPricesMissingError('possibly delisted; no price data found  (1h 2023-01-01 -> 2023-12-31) (Yahoo error = "1h data not available for startTime=1672549200 and endTime=1703998800. The requested range must be within the last 730 days.")')


+--------+-------------+---------+--------+-------+--------+----------+
| Date   | Adj Close   | Close   | High   | Low   | Open   | Volume   |
|--------+-------------+---------+--------+-------+--------+----------|
+--------+-------------+---------+--------+-------+--------+----------+


In [20]:
df["rsi_14"] = ta.rsi(df["Close"], length=14)
df["rsi_28"] = ta.rsi(df["Close"], length=28)
df["rsi_7"] = ta.rsi(df["Close"], length=7)

macd = ta.macd(df["Close"])
# df = df.join(macd)
df["macd"] = macd["MACD_12_26_9"]

df["ema_20"] = ta.ema(df["Close"], length=20)
df["ema_50"] = ta.ema(df["Close"], length=50)

# Stochastic Oscillator
stoch = ta.stoch(df["High"], df["Low"], df["Close"])
df["stoch_k"] = stoch["STOCHk_14_3_3"]
df["stoch_d"] = stoch["STOCHd_14_3_3"]

df["roc"] = ta.roc(close=df["Close"], length=10)

adx = ta.adx(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    length=14
)
df["adx"]  = adx["ADX_14"]
df["di_plus"]  = adx["DMP_14"] 
df["di_minus"] = adx["DMN_14"]  

df["atr"] = ta.atr(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    length=14
)

df["close_pos"] = (df["Close"] - df["Low"]) / (df["High"] - df["Low"])

df["body_range_ratio"] = (df["Close"] - df["Open"]).abs() / (df["High"] - df["Low"])


df["returns"] = df["Close"].pct_change()
df['direction'] = np.where(df['returns'] > 0, 1, 0)

df.dropna(inplace=True)

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

df.to_csv(PATH)

TypeError: 'NoneType' object is not subscriptable